In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
import pandas

'''
Authenticate
Authenticates your credentials and creates a client.
pip install azure-cognitiveservices-vision-computervision
'''
subscription_key = "91d0454ee5244ea88056d7e79b73b9d2"
endpoint = "https://computerkun.cognitiveservices.azure.com/"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

'''
OCR: Read File using the Read API, extract text - remote
This example will extract text in an image, then print results, line by line.
This API call can also extract handwriting style text (not shown).
'''
print("===== Read File - remote =====")
# Get an image with text
read_image_url = "https://stat.ameba.jp/user_images/20140221/16/koumuinsiken-yobikou/86/3b/p/o0471068512853433393.png"

# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

textweb = []
# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            textweb.append(line.text)
            print(line.bounding_box)
print()

===== Read File - remote =====


ClientRequestError: Error occurred in request., ConnectionError: HTTPSConnectionPool(host='computerkun.cognitiveservices.azure.com', port=443): Max retries exceeded with url: /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2e97181160>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))

In [11]:
'''
OCR: Read File using the Read API, extract text - local
This example extracts text from a local image, then prints results.
This API call can also recognize remote image text (shown in next example, Read File - remote).
'''
print("===== Read File - local =====")

images_folder = "/workspace/products"
# Get image path
read_image_path = os.path.join (images_folder, "image2.jpeg")
# Open the image
read_image = open(read_image_path, "rb")

# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(10)

textlocal = []
# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            textlocal.append(line.text)
            print(line.bounding_box)
print()
'''
END - Read File - local
'''
df_local = pd.DataFrame(textlocal, columns = ["sentence"])
df_web = pd.DataFrame(textweb, columns = ["sentence"])
display(df_web)
print("End of Computer Vision quickstart.")

===== Read File - local =====


NewConnectionError: <urllib3.connection.HTTPSConnection object at 0x7fd6d552f1c0>: Failed to establish a new connection: [Errno -5] No address associated with hostname